## Load trained models, and take new metric of test performance - how often correct max or mins are selected

In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
# funcs stolen from nyu deep learning course
from sequential_tasks import pad_sequences, to_categorical
import os
import random
import torch
from load_data_funs import load_data, gen_batch_data_fixations_choice, gen_batch_data_fixations_only, gen_batch_data_choice_only

on_cluster=True
if on_cluster:
    sim_data_path = '/scratch/gpfs/erussek/RNN_project/optimal_fixation_sims'
    human_data_path = '/scratch/gpfs/erussek/RNN_project/human_trials.json'
else:
    sim_data_path = '/Users/evanrussek/Dropbox/Griffiths_Lab_Stuff/Data/RNNs/optimal_fixation_sims'
    human_data_path = '/Users/evanrussek/Dropbox/Griffiths_Lab_Stuff/Data/RNNs/human_trials.json'

In [2]:
# load data train and test sets...
train_data_sim, test_data_sim, human_data = load_data(sim_data_path, human_data_path, this_seed=job_idx)

NameError: name 'job_idx' is not defined

In [ ]:
# compoute correlation with held-out test data... 
def test_record_each_output(model, test_sim_data, device, batch_size, n_total_seq, gen_batch_data,out_idx, choice_only=False, human_data=False):
    # Set the model to evaluation mode. This will turn off layers that would
    # otherwise behave differently during training, such as dropout.
    
    # print(choice_only)
    model.eval()

    # Store the number of sequences that were classified correctly
    # num_correct = 0

    n_batches = int(np.round(n_total_seq/batch_size));

    output_all = np.zeros((0,3))
    target_all = np.zeros((0,3))

    # A context manager is used to disable gradient calculations during inference
    # to reduce memory usage, as we typically don't need the gradients at this point.
    with torch.no_grad():
        for batch_idx in range(n_batches):
            data, target = gen_batch_data(batch_size, batch_idx, test_sim_data, human_data = human_data)
            data, target = torch.from_numpy(data).float().to(device), torch.from_numpy(target).long().to(device)

            output = model(data)
            # Pick only the output corresponding to last sequence element (input is pre padded)
            if not choice_only:
                output = output[:, -out_idx, :]
                target = target[:,-out_idx,:]

            output_all = np.concatenate((output_all, output.numpy()))
            target_all = np.concatenate((target_all, target.numpy()))

    return (output_all, target_all)


In [ ]:
def load_model(to_save_folder, train_setting, job_idx):
    
    model_full_file_name = os.path.join(to_save_folder, 'model_train_setting_{}_job_{}'.format(train_setting,job_idx))
    trained_model = torch.load(model_full_file_name)

    return trained_model



In [ ]:



train_setting=0
job_idx=0

on_cluster = True
# set up folder to save results
if on_cluster:
    to_save_folder = '/scratch/gpfs/erussek/RNN_project/train_on_sim_results'
else:
    to_save_folder = '/Users/evanrussek/Dropbox/Griffiths_Lab_Stuff/Code/RNNs/train_on_sim_results'

def get_pct_max_min(trained_model, test_data_sim, train_setting, n_back):

    train_data_funcs = [gen_batch_data_fixations_choice, gen_batch_data_fixations_only, gen_batch_data_choice_only]
    this_data_func = train_data_funcs[train_setting]

    # train on a 1 mil. examples, generate learning curves... 
    batch_size  = 32
    n_total_seq = 1.5e6
    n_batches = int(np.round(n_total_seq/batch_size));
    n_tests = int(np.ceil(n_batches/200)) - 1
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    n_seq_test = 1000
    
    choice_only=False
    if train_setting==2:
        choice_only=True
        
    output_all_LSTM, target_all_LSTM = test_record_each_output(trained_model, test_data_sim, device, batch_size, n_seq_test,this_data_func, n_back,choice_only=choice_only, human_data=human_data)

    output_max_item = output_all_LSTM.argmax(axis=1)
    target_max_item = target_all_LSTM.argmax(axis=1)
    pct_correct_max = np.sum(output_max_item == target_max_item)/len(output_max_item)

    output_min_item = output_all_LSTM.argmin(axis=1)
    target_min_item = target_all_LSTM.argmin(axis=1)
    
    print(output_min_item)
    
    pct_correct_min = np.sum(output_min_item == target_min_item)/len(output_min_item)
    
    return pct_correct_max, pct_correct_min


get_pct_max_min(trained_model, test_data_sim, 1, 1)

In [ ]:
def load_model_and_get_pct_max_min(to_save_folder, train_setting, job_idx, n_back=1):
    trained_model = load_model(to_save_folder, train_setting, job_idx)
    return get_pct_max_min(trained_model, test_data_sim, train_setting, n_back)


all_jobs = np.arange(50)
if train_setting < 2:
    bad_jobs = []
else:
    bad_jobs = [1,10]

good_jobs = np.delete(all_jobs,bad_jobs);

n_jobs = len(good_jobs)
max_min_res0 = np.zeros((len(good_jobs),2))
max_min_res1 = np.zeros((len(good_jobs),2))
max_min_res2 = np.zeros((len(good_jobs),2))

for i in range(n_jobs):
    print(i, end=' ')
    max_min_res0[i,:] = load_model_and_get_pct_max_min(to_save_folder, 0, good_jobs[i], n_back=1)  
    #max_min_res1[i,:] = load_model_and_get_pct_max_min(to_save_folder, 1, good_jobs[i], n_back=1)
    #max_min_res2[i,:] = load_model_and_get_pct_max_min(to_save_folder, 2, good_jobs[i], n_back=0)   

In [ ]:
np.mean(max_min_res1,0)

In [85]:
np.mean(max_min_res2,0)

array([0.72983871, 0.49431452])